<a href="https://colab.research.google.com/github/SIVASKANDAN-P/PYTHON-CODE-HINT/blob/main/Code_solution_experimentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install -U datasets
!pip install bert_score
!pip install peft
!pip install accelerate
!pip install bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.3 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.44.1
    Uninstalling bitsandbytes-0.44.1:
      Successfully uninstalled bitsandbytes-0.44.1


In [2]:
#importing necessary libraries
from datasets import load_dataset, load_dataset_builder
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from google.colab import drive
drive.mount('/content/drive')
from bert_score import score
from datetime import datetime
import os
import sys

import torch
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
#    prepare_model_for_int8_training,
    prepare_model_for_kbit_training,
    set_peft_model_state_dict
)
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from transformers import T5Tokenizer, T5ForConditionalGeneration


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Load dataset
dataset = load_dataset("openbmb/UltraInteract_sft",split="train")
#convert to pandas dataframe
data = pd.DataFrame(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.0k [00:00<?, ?B/s]

0000_sft.parquet:   0%|          | 0.00/171M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/288579 [00:00<?, ? examples/s]

In [ ]:
#load dataset using dataset builder
#load_dataset_builder("openbmb/UltraInteract_sft")
#printing the featured in dataset
#ds_builder.info.features

In [4]:
#number of fields in dataframe
data.columns

Index(['task', 'dataset', 'instruction', 'response', 'id', 'parent_id'], dtype='object')

In [5]:
#The number of examples we have for each task
data.task.value_counts()

,count
task,
Coding,114826
Math_PoT,83913
Math_CoT,78349
Logic,11491


Adding colums for getting the length of instruction and response column for getting idea on required context length in LLM. And selecting examples only satisfying context length

In [6]:
def count_words(text):
    words = text.split()  # Split the string by whitespace
    return len(words)

In [7]:
#Adding colums for getting the length of instruction column
data['ins-len'] = data['instruction'].apply(lambda x: len(x.split()))
#Adding colums for getting the length of Response column
data['res-len'] = data['response'].apply(lambda x: len(x.split()))
#Adding colums for sum of the length of instruction and response columns
data['max-len'] = data['res-len'] + data['ins-len']

In [23]:
#printing number of records with max length less than 300
data_max_300 = data[data['max-len'] < 300]
print(data[data['max-len'] < 300].task.value_counts())

task
Math_PoT    62197
Math_CoT    61845
Coding      21110
Logic        2273
Name: count, dtype: int64


In [9]:
#max len in the full dataset
data['max-len'].max()

1915

In [24]:
#max length in filtred dataset
data_max_300['max-len'].max()

299

In [25]:
print(data_max_300.task.value_counts())

task
Math_PoT    62197
Math_CoT    61845
Coding      21110
Logic        2273
Name: count, dtype: int64


In [ ]:
#Loading the tokenizer for model
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [22]:
#save dataset to drive
data.to_csv(r'/content/drive/MyDrive/dataset/ultrainteract.csv')

In [26]:
#save dataset to drive
data_max_300.to_csv(r'/content/drive/MyDrive/dataset/ultrainteract_maxlen_300.csv')


In [21]:
!cd /content/drive/MyDrive/dataset

In [39]:
#dataset = load_dataset("csv", data_files="/content/drive/MyDrive/dataset/ultrainteract_maxlen_300.csv")

In [38]:
#dataset

In [37]:
# Extract the train and test datasets from the split
#train_dataset = dataset["train"]
#test_dataset = dataset["test"]

In [4]:
dataset = pd.read_csv("/content/drive/MyDrive/dataset/ultrainteract_maxlen_300.csv")

In [5]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.1)

In [3]:
model1 = "google/flan-t5-large"
model2 = "google/flan-t5-small"
model3 = "google/flan-t5-base"
model4 = "google/flan-t5-xl"
model5 = "google/flan-t5-xxl"
model6 = "codellama/CodeLlama-7b-hf"

In [4]:
base_model = model1

loading t5 models using cpu

In [ ]:

tokenizer = T5Tokenizer.from_pretrained(base_model)
model = T5ForConditionalGeneration.from_pretrained(base_model)

input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/9.60G [00:00<?, ?B/s]

KeyboardInterrupt: 

loading t5 models using GPU

In [8]:
# pip install accelerate

tokenizer = T5Tokenizer.from_pretrained(base_model)
model = T5ForConditionalGeneration.from_pretrained(base_model, device_map="auto")

input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> Wie alte sind Sie?</s>


In [ ]:
model.num_parameters()/ 1000000000

0.78315008

In [18]:
!pip install -U bitsandbytes

In [5]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4")

In [6]:
model_4bit = T5ForConditionalGeneration.from_pretrained(
    base_model,
    quantization_config=quantization_config
)
tokenizer = T5Tokenizer.from_pretrained(base_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.
You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing 

In [7]:
model = prepare_model_for_kbit_training(model_4bit)

In [8]:
print(model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear4bit(in_features=1024, out_features=1024, bias=False)
              (k): Linear4bit(in_features=1024, out_features=1024, bias=False)
              (v): Linear4bit(in_features=1024, out_features=1024, bias=False)
              (o): Linear4bit(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear4bit(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear4bit(in_features=1024, out_features=2

In [9]:
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules="all-linear",  # To apply LoRA to all the linear layers
    lora_dropout=0.05,
    bias="none",                  #since for other when adopter is removed model will not perfor the same
    task_type="CAUSAL_LM"         #since it is text generation task
)

model = get_peft_model(model, config)

In [ ]:
def prompt_instruction_format(sample):
  return f"""### Instruction:
    Use the Task below and the Input given to write the Response:

    ### Task:
    Summarize the Input

    ### Input:
    {sample['dialogue']}

    ### Response:
    {sample['summary']}
    """

In [ ]:
#model size in billion
model_size[base_model] = model.num_parameters()/ 1000000000

In [ ]:
model_size

In [ ]:
model_size_df = pd.DataFrame(list(model_size.items()),columns = ['Model','Size'])

In [ ]:
model_size_df.to_csv(r'model_size_df.csv')

In [ ]:
# prompt: list the directory current

!cd drive
!cd MyDrive
!cd ULTRAINTERACT


/bin/bash: line 1: cd: MyDrive: No such file or directory
/bin/bash: line 1: cd: ULTRAINTERACT: No such file or directory


In [ ]:
!ls


drive  sample_data
